In [ ]:
from __future__ import division
import gym
import torch
import random
import numpy as np
import torch
from PIL import Image
import torch
import torch.nn as nn
from collections import namedtuple
from collections import deque
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
from matplotlib.pyplot import imshow
from PIL import Image
from wrappers import make_atari, wrap_deepmind, wrap_pytorch
import queue
from torch import optim
import matplotlib.pyplot as plt

# Model

In [ ]:
class QNet(torch.nn.Module):
    def __init__(self,obs_shape,act_shape):
        super(QNet, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(7*7*64,512)
        self.fc2 = nn.Linear(512,1)
        self.fc3 = nn.Linear(7*7*64,512)
        self.fc4 = nn.Linear(512,act_shape)
    def forward(self, x):
        #Conv
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = x.view(x.shape[0],-1)
        
        #Fc
        x1 = x
        x = self.fc1(x)
        x = self.relu(x)
        v = self.fc2(x)
        
        x1 = self.fc3(x1)
        x1 = self.relu(x1)
        adv = self.fc4(x1)
        
        q_s_a = v + adv - adv.mean()
        
        return q_s_a

In [ ]:
def eps_greedy(epsilon,state,net):
    if(np.random.random()<epsilon):
        action = np.random.randint(ACT_SHAPE)
    else:
        qvalues = net(state)
        action = torch.argmax(qvalues).item()
    return action    

In [ ]:
class ReplayBuffer(object):
    def __init__(self,maxsize):
        self.q = deque(maxlen = maxsize)
        self.maxsize = maxsize
    def add(self,x):
        self.q.append(x)
        if(len(self.q)==self.maxsize):
            self.q.popleft()
    def getSize(self):
        return len(self.q)
    def sample(self,size):
        batch = random.sample(list(self.q),size)
        state,action,reward,next_state,done = map(list, zip(*batch))
        return state,action,reward,next_state,done
           

In [ ]:
class SegmentSumTree:
    def __init__(self, capacity):
        self.size = capacity 
        #n leaves + n-1 internal = 2n-1, capacity of a tree
        self.tree = np.zeros((2*capacity-1), dtype = np.float32)
        #leaf nodes having actual values 
        self.transit_data_buffer =  np.array([None] * self.size) 
        self.is_full = False
        self.max_prior_val = 1
        self.num_entries = 0 
        self.idx = 0 
    
    
    def append(self, priority_val, transit_data):
        #self.max_prior_val = max(priority_val, self.max_prior_val)
        self.transit_data_buffer[self.idx] = transit_data  
        self.update(self.idx + self.size - 1, priority_val) 
        self.idx = (self.idx + 1) % self.size 
        self.num_entries += 1
        if self.num_entries > self.size:
            self.num_entries = self.size
        self.is_full = self.is_full or self.idx == 0  
    
    
    def propagate(self, index, priority_val):
        parent_idx = self.getParentIdx(index)
        left_node_idx = 2 * parent_idx + 1
        right_node_idx = 2 * parent_idx + 2
        self.tree[parent_idx] = self.tree[left_node_idx] + self.tree[right_node_idx]
        if parent_idx != 0: 
            self.propagate(parent_idx, priority_val)
            
        
    def update(self, index, priority_val):
#         print('priority',priority_val)
        self.tree[index] = priority_val
        self.max_prior_val = max(priority_val, self.max_prior_val)
        self.propagate(index, priority_val)
       
    
    def getParentIdx(self, index):
        return (index - 1)//2
        
            
        
    def search(self, value):
        idx = self.retrieve(0, value)  # Search for index of item from root
        data_index = idx - self.size + 1
        return (self.tree[idx], data_index, idx)
        
    
    def retrieve(self, idx, value):
        left_node_idx = 2 * idx + 1
        right_node_idx = 2 * idx + 2
        
        if left_node_idx >= len(self.tree):
            return idx
        
        elif value <= self.tree[left_node_idx]:
            return self.retrieve(left_node_idx, value)
            
        else:
            return self.retrieve(right_node_idx, value - self.tree[left_node_idx])
        
    
    def getNumEntries(self):
        return self.num_entries
     

    def getTotal(self):
        return self.tree[0]
        
    def getSize(self):
        return self.size
    
    
    def getMaxPriorValue(self):
        return self.max_prior_val
    
    
    def getPriorties(self):
        return self.priorities
    
    
    def getTree(self):
        return self.tree
    
    
    def getDataByIdx(self, idx):
        return self.transit_data_buffer[idx % self.size]
    
    
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state','done'))

class PrioritizedReplayBuffer:
    def __init__(self,capacity = 1000000, batch_size = 32, gamma = 0.99, multi_step = 3, 
                beta = 0.4, alpha = 0.5):
        self.capacity = capacity
        self.discount = gamma
        self.n = multi_step
        self.beta = beta   
        self.alpha = alpha
        self.transit_buffer = SegmentSumTree(capacity) 
               

    def add(self,state,action,reward,next_state,done):
        
        self.transit_buffer.append(self.transit_buffer.max_prior_val, Transition(state, action, reward, next_state, done))  
        
        
    def sample(self,k): 
        batch, idxs, priorities = [], [], []
        root_total_priority = self.transit_buffer.getTotal()
        segment = root_total_priority / k
        
#         priority_exponent_weight = (1 - self.beta) / (TMAX - TMIN)
        priority_exponent_weight = (1 - self.beta) / 1000
        self.beta = np.min([1.0, self.beta + priority_exponent_weight])
#         print('total priority',root_total_priority)
        
        for i in range(k):
            a = segment * i
            b = segment * (i + 1)
            samp = random.uniform(a, b)
#             print('sample : ',samp)
            priority, data_idx, idx = self.transit_buffer.search(samp)
#             print('data idx',data_idx)
            data = self.transit_buffer.getDataByIdx(data_idx)
            priorities.append(priority)
            batch.append(data)
            idxs.append(idx)
        #print('*'*100)
        sum_priority = np.power(self.transit_buffer.tree[:self.transit_buffer.getNumEntries()],self.alpha)
        sum_priority = sum_priority.sum()
        sampling_probabilities = np.power(priorities, self.alpha) / sum_priority #root_total_priority
        #
        
        #print(sampling_probabilities)
        
        #compute importance sampling 
        weights = np.power(self.transit_buffer.getNumEntries() * sampling_probabilities, -self.beta)
        weights /= weights.max()
        
#         try:
        states, actions, rewards, next_states, dones = zip(*batch)
#         except:
#             print('error in ',batch)
        
        states,actions, rewards, next_states, dones = list(states), list(actions), list(rewards), \
                                                    list(next_states), list(dones) 
        states = torch.cat([x for x in states]).cuda()
        next_states = torch.cat([x for x in next_states]).cuda()
        actions = torch.Tensor(actions).long().cuda()
        dones = np.array(dones).astype(int)
        dones = torch.Tensor(dones).cuda()
        rewards = torch.Tensor(rewards).cuda()
        #priorities = torch.Tensor(priorities).cuda()
        weights = torch.Tensor(weights).cuda()
        
        return weights, idxs, states, actions, rewards, next_states, dones       
        
        
    def updatePriorities(self, indexes, priorities):
        #priorities = np.power(priorities, self.alpha)
        for idx, priority_val in zip(indexes, priorities):
            self.transit_buffer.update(idx, priority_val)
        
        
    def getSize(self):
        return self.transit_buffer.getNumEntries()
        

# Loss function

In [ ]:
def compute_loss(size):
    weights,idx,current_state,action,reward,next_state,done = buffer.sample(size)
    
    
    qvalues = net(current_state)
    qvalues_next = net(next_state)
    target_net.eval()
    qvalues_target = target_net(next_state)
    
    #Q(s,a)
    q_a = qvalues.gather(1, action.unsqueeze(1)).squeeze(1)
    
    #Selecting action for target network
    selected = qvalues_next.max(1)[1]
    #Q'(s',argmax Q(s',a))
    q_a_target = qvalues_target.gather(1,selected.unsqueeze(1)).squeeze(1)
    
    #Computing target value
    target = reward + GAMMA * q_a_target * (1 - done)
    L = (target - q_a)#.pow(2)*weights
    
#     print('priority : ',weights)
    new_priorities =  torch.abs(L) + edge_epsilon
#     L = torch.clamp(L,-1,1)
    L = L.pow(2) * weights 
    L = L.mean()
    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    new_priorities = new_priorities.cpu().detach().numpy()
    buffer.updatePriorities(idx,new_priorities)
#     print("Loss : ",L)
    
    
    return L

In [ ]:
def update_target():
    target_net.load_state_dict(net.state_dict())

In [ ]:
def epsilon_decay(ep):
    e = .01 + .99*np.exp(-ep/30000)
    return e

In [ ]:
def addreward(id,item,filename):
    f=open(filename,'a+')
    f.write(str(id)+' '+str(item)+' '+'\n')
    f.close()

In [ ]:
def addloss(id,loss,filename):
    f=open(filename,'a+')
    f.write(str(id)+' '+str(loss.item())+' '+'\n')
    f.close()

# Training

In [ ]:
# env = gym.make('PongNoFrameskip-v4')
env    = make_atari('PongNoFrameskip-v4')
# env    = make_atari('BankHeist-v0')
# env    = make_atari('Pong-v0')
env    = wrap_deepmind(env)
env    = wrap_pytorch(env)

print(env.observation_space)
print(env.action_space)


In [ ]:
seed = 123
np.random.seed(seed)
torch.manual_seed(np.random.randint(1, 10000)) 
net = QNet(env.observation_space.shape,env.action_space.n)
net = net.cuda()
target_net = QNet(env.observation_space.shape,env.action_space.n)
target_net = target_net.cuda()
update_target()
optimizer = optim.Adam(net.parameters(), lr=0.0000625,eps=1.5e-4)

In [ ]:
lossfile = './prioritized-logs/losses.csv'
rewardsfile = './prioritized-logs/rewards.csv'
if os.path.exists(lossfile):
    os.remove(lossfile)
if os.path.exists(rewardsfile):
    os.remove(rewardsfile)

In [ ]:
ITERATIONS = 5e6
epsilon = 0.99
OBS_SHAPE = env.observation_space.shape
ACT_SHAPE = env.action_space.n
REPLAY_SAMPLE = 20000
BATCH_SIZE = 32
GAMMA = 0.99
T_upd = 10000
TMAX = 50e6
TMIN = 20e3
edge_epsilon = 1e-5
save_model_interval = 1e6
loss_log_interval = 1e3
episode_reward_interval = 10

In [ ]:
#buffer = ReplayBuffer(10000)
buffer = PrioritizedReplayBuffer(100000)
episode_reward = 0
state = env.reset()
state = torch.Tensor(state).cuda()
state = state.unsqueeze(0)
count = 0
loss_count = 0
rewards_dict = {}
loss_dict = {}
losses = []
rewards = [] 
for i in tqdm(range(1,ITERATIONS+1)):
    epsilon = epsilon_decay(i)
    action = eps_greedy(epsilon,state,net)
    next_state, reward, done, info = env.step(action)
    next_state = torch.Tensor(next_state).unsqueeze(0).cuda()
#     print('state input in buffer ',state.shape)
#     print('state input in buffer ',next_state.shape)
    episode_reward+=reward
#     reward = max(min(reward,1),-1)
    buffer.add(state,action,reward,next_state,done)
    state = next_state
    if(buffer.getSize()>REPLAY_SAMPLE and i%4==0):
        loss = compute_loss(BATCH_SIZE)
#         losses.append(loss)
#         if(i%100==0):
#             print("Loss",loss.item())
#             addloss(i,loss,lossfile)
        if i % loss_log_interval == 0:
            loss_dict[i] = {}
            loss_dict[i]['loss'] = loss.item()
            loss_count += 1
            if loss_count == 10: 
                loss_df = pd.DataFrame.from_dict(data = loss_dict, orient = 'index').reset_index()
                if not os.path.exists(lossfile):
                    loss_df.to_csv(lossfile,index=None, header='column_names')
                else: # else it exists so append without writing the header
                    loss_df.to_csv(lossfile, mode='a',index=None, header=False)
                loss_dict = {}
                loss_count = 0
    if done:
        count += 1
        rewards.append(episode_reward)
       
        if count%50 == 0:
                print(i,count,episode_reward) 
        if count%episode_reward_interval == 0: 
                rewards_df = pd.DataFrame.from_dict(data = rewards_dict, orient = 'index').reset_index()
                if not os.path.exists(rewardsfile):
                    rewards_df.to_csv(rewardsfile,index=None, header='column_names')
                else: # else it exists so append without writing the header
                    rewards_df.to_csv(rewardsfile, mode='a',index=None, header=False)
                rewards_dict = {}
        episode_reward = 0 
        state = env.reset()
        state = torch.Tensor(state).cuda()
        state = state.unsqueeze(0)
    if(i%T_upd==0):
        update_target()

    if(i%save_model_interval == 0):
        torch.save(net.state_dict(),'./prioritized-logs/prioritized-model' + str(i) + '.pth')
        torch.save(target_net.state_dict(),'./prioritized-logs/prioritized-model-target' + str(i) + '.pth')